In [ ]:
#Importamos las librerias
import pandas as pd
from rapidfuzz import process, fuzz

# --- Cargar archivos ---
# df_festividades: dataset con festividades por departamento
# df_videos: dataset con videos turísticos relacionados a festividades
df_festividades = pd.read_csv("../Festividades/festividades_departamentos_peru.csv")
df_videos = pd.read_csv("../Videos/videos_por_festividad_y_departamento.csv")

# --- Renombrar columna ---
df_videos.rename(columns={'Festividad': 'Festividades'}, inplace=True)

# --- Normalizar texto ---
# Función auxiliar para estandarizar los textos:
# - Convierte a mayúsculas
# - Elimina espacios adicionales
# - Maneja valores nulos

def limpiar(texto):
    if pd.isna(texto):
        return ''
    return str(texto).upper().strip()

# Aplicar normalización a 'Departamento' y 'Festividades' en ambos datasets
for col in ['Departamento', 'Festividades']:
    df_festividades[col] = df_festividades[col].apply(limpiar)
    df_videos[col] = df_videos[col].apply(limpiar)

# --- Filtrar videos válidos ---
# Se eliminan registros con URL vacías o mal formateadas
df_videos = df_videos[df_videos['URL'].notna()]
df_videos = df_videos[df_videos['URL'].str.startswith("http")]

# --- Crear claves ---
# La clave combina 'Departamento' y 'Festividades'
df_festividades['Clave'] = df_festividades['Departamento'] + "|" + df_festividades['Festividades']
df_videos['Clave'] = df_videos['Departamento'] + "|" + df_videos['Festividades']

# --- Quitar duplicados por clave ---
df_videos_unique = df_videos.drop_duplicates(subset='Clave', keep='first')

# --- Diccionarios auxiliares ---
# Diccionario principal con claves de festividad y su video asociado
video_dict = df_videos_unique.set_index('Clave')[['Título del Video', 'URL']].to_dict(orient='index')
videos_por_dpto = df_videos_unique.groupby('Departamento').first()[['Título del Video', 'URL']].to_dict(orient='index')
claves_disponibles = set(video_dict.keys())

# --- Fuzzy matching ajustado a 80% ---
def buscar_mejor_video(clave_festividad):
    matches = process.extract(
        clave_festividad,
        list(claves_disponibles),
        scorer=fuzz.token_sort_ratio,
        limit=1
    )
    if matches and matches[0][1] >= 80:  # más tolerante
        return matches[0][0]
    return None

# --- Asignación ---
asignadas = set()
resultados = []

# Recorrer cada fila de festividades
for _, row in df_festividades.iterrows():
    clave_fest = row['Clave']
    clave_video = buscar_mejor_video(clave_fest)
    depa = row['Departamento']

    if clave_video and clave_video not in asignadas:
        # Si se encontró un video por coincidencia fuzzy y no ha sido asignado antes
        info = video_dict[clave_video]
        asignadas.add(clave_video)
    else:
        # Si no se encuentra coincidencia fuzzy, usar el primer video disponible del departamento
        info = videos_por_dpto.get(depa, {'Título del Video': 'No encontrado', 'URL': 'N/A'})

        # Guardar la fila con toda la información
    resultados.append({
        'Departamento': row['Departamento'],
        'Fechas': row['Fechas'],
        'Lugar': row['Lugar'],
        'Festividades': row['Festividades'],
        'Título del Video': info['Título del Video'],
        'URL': info['URL']
    })

# --- DataFrame final ---
df_final = pd.DataFrame(resultados)

# --- Verificación ---
print(f"\n✅ Total de filas: {len(df_final)}") 

df_final




✅ Total de filas: 777


,Departamento,Fechas,Lugar,Festividades,Título del Video,URL
0,AMAZONAS,Febrero (Movible),Todo el departamento,CARNAVALES.,Carnaval de Bagua - Amazonas - Perú (2023),https://www.youtube.com/watch?v=0SrFkcucWIw&pp...
1,AMAZONAS,1-6 de mayo,Todo el departamento,FIESTA DE LAS CRUCES.,Así se celebra la fiesta de las Cruces en Caja...,https://www.youtube.com/watch?v=F4bcK2NQ0qM&pp...
2,AMAZONAS,24 de junio,Bongará,FIESTA DE SAN JUAN.,Así se vive la tradicional fiesta de “San Juan...,https://www.youtube.com/watch?v=PeYmpXJOV-M&pp...
3,AMAZONAS,25-29 de junio,Todo el departamento,FESTIVIDAD DE SAN PEDRO Y SAN PABLO.,Ancón celebra con fe y alegría a San Pedro y S...,https://www.youtube.com/watch?v=-gUd0esRyVg&pp...
4,AMAZONAS,Movible,Chachapoyas,SEMANA TURÍSTICA.,Raymi Llaqta 2025: Todo sobre la semana turíst...,https://www.youtube.com/watch?v=_XC1Ecnl304&pp...
...,...,...,...,...,...,...
772,UCAYALI,15-21 de febrero,Ucayali,FIESTAS DE CARNAVALES,Carnaval en Ucayali: Corso carnavalesco llenó ...,https://www.youtube.com/watch?v=SXcsBW_mBnw&pp...
773,UCAYALI,07 de enero,Tumbes,ANIVERSARIO DE LA DECLARACIÓN DE INDEPENDENCIA...,Aniversario de Tumbes: la región celebra 204 a...,https://www.youtube.com/watch?v=E_Gaa9SL37c&pp...
774,UCAYALI,13 de mayo,Pucallpa,FESTIVIDAD DE LA VIRGEN MARÍA O LA PEREGRINA,ALERTA MUNDIAL! Asís Tiene la Clave para Deten...,https://www.youtube.com/shorts/F1zIqI3pXx8
775,UCAYALI,24 de junio,Región Amazónica,FIESTA DE SAN JUAN,Así se vivió la Fiesta de San Juan en Pucallpa...,https://www.youtube.com/watch?v=EFBMCGj9sKg&pp...


In [48]:
# --- Guardar archivo ---
df_final.to_excel("festividades_lugares_contenido.xlsx", index=False)
print("📁 Archivo generado: festividades_lugares_contenido.xlsx")

📁 Archivo generado: festividades_lugares_contenido.xlsx
